# Husky Enrollment by town

Analysis started Friday, March 18, 2016.

## 0 Data import

### 0.A Import Husky sheet

In [1]:
import pandas as pd

## Load HUSKY A tab, skipping header lines
husky = pd.read_excel("HUSKYA and Medicaid by Town with %.xls",
                           sheetname=0,
                           skiprows=6, 
                           names=["town","population","husky_enrollment","husky_pct_enrollment"])

## (Uncomment below) Check that their calculated enrollment percent matches my own (should equal zero)
#husky["calculated_pct"] = husky["enrollment"] / husky["population"]
#husky["calculated_pct"].sum() - husky["pct_enrollment"].sum()


## Add a column for a human-friendly percent
husky["husky_pct"] = husky["husky_pct_enrollment"] * 100

## Drop origin percent col
husky = husky.drop(["husky_pct_enrollment"], 1)

## (Uncomment below) Describe the dataset
#husky.describe()

## (Uncomment below) Count rows
husky.count()

town                170
population          170
husky_enrollment    170
husky_pct           170
dtype: int64

### 0.b Import Medicaid sheet

In [2]:
## Load Sheet 2
medicaid = pd.read_excel("HUSKYA and Medicaid by Town with %.xls",
                         sheetname="Medicaid Enrollment by Town",
                         skiprows=4,
                         names=["town","population","med_enrollment","drop","med_pct"])

## fourth column is empty
medicaid = medicaid.drop("drop",1)

## drop null rows
medicaid = medicaid[medicaid["population"].notnull()]


## Check that their percentages are correct
#medicaid["calculated_pct"] = medicaid["med_enrollment"] / medicaid["population"]
#medicaid["calculated_pct"].sum() - medicaid["med_pct"].sum()

## Add a human-readable percent column
medicaid["med_pct"] = medicaid["med_pct"] * 100

## Drop original percent column
#medicaid = medicaid.drop("med_pct", 1)

## (Uncomment below) Count rows
medicaid.count()

town              170
population        170
med_enrollment    170
med_pct           170
dtype: int64

### Merge sheets

In [3]:
combined = medicaid.merge(husky,on=["town","population"])

## Count rows. Should have the same # no of rows as medicaid and husky
combined.count()

town                169
population          169
med_enrollment      169
med_pct             169
husky_enrollment    169
husky_pct           169
dtype: int64

In [4]:
## Something went wrong. Let's find the row that didn't synch

## Rows in husky but not in combined
#husky[~husky["town"].isin(combined["town"])]

## Rows in medicaid but not in combined
#medicaid[~medicaid["town"].isin(combined["town"])]


In [5]:
## North stonington is the problem. It's entered both as "No. Stonington" and "North Stonington"
medicaid = medicaid.replace("No. Stonington", "North Stonington")


## Try merge again
combined = medicaid.merge(husky,on=["town","population"])

## Count rows. Should have the same # no of rows as medicaid and husky
combined.count()

town                170
population          170
med_enrollment      170
med_pct             170
husky_enrollment    170
husky_pct           170
dtype: int64

In [6]:
## Good! Moving on...

## 1 Towns with the highest enrollment

Canaan has a population of 1,195, so the percentage each resident accounts for is significantly higher than the other places on the list. We'll look just at towns with populations over 5,000

### 1.a Top medicaid enrollment

Canaan has a population of 1,195, so the percentage each resident accounts for is significantly higher than the other places on the list.

In [7]:
combined[combined["population"] > 5000].sort_values("med_pct", ascending=False).head(10)

,town,population,med_enrollment,med_pct,husky_enrollment,husky_pct
63,Hartford,124705,71755,57.539794,39917,32.009142
150,Waterbury,109307,55828,51.074497,33099,30.280769
88,New Britain,72878,34049,46.720547,20398,27.989242
14,Bridgeport,147612,66854,45.290356,40457,27.407663
92,New Haven,130282,56711,43.529421,31913,24.495326
93,New London,27374,11716,42.799737,6830,24.950683
162,Windham,25005,10163,40.643871,5682,22.723455
103,Norwich,40178,15467,38.496192,9106,22.664145
41,East Hartford,51033,19261,37.742245,11487,22.508965
79,Meriden,60293,22091,36.639411,13079,21.692402


### 1.b Top husky enrollment

In [8]:
combined[combined["population"] > 5000].sort_values("husky_pct", ascending=False).head(10)

,town,population,med_enrollment,med_pct,husky_enrollment,husky_pct
63,Hartford,124705,71755,57.539794,39917,32.009142
150,Waterbury,109307,55828,51.074497,33099,30.280769
88,New Britain,72878,34049,46.720547,20398,27.989242
14,Bridgeport,147612,66854,45.290356,40457,27.407663
93,New London,27374,11716,42.799737,6830,24.950683
92,New Haven,130282,56711,43.529421,31913,24.495326
162,Windham,25005,10163,40.643871,5682,22.723455
103,Norwich,40178,15467,38.496192,9106,22.664145
41,East Hartford,51033,19261,37.742245,11487,22.508965
79,Meriden,60293,22091,36.639411,13079,21.692402


In [9]:
# Add a column for the share of the husky program 
combined["husky_share"] = combined["husky_enrollment"] * 100 / combined["med_enrollment"]

combined.describe()

,population,med_enrollment,med_pct,husky_enrollment,husky_pct,husky_share
count,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000
mean,42313.847059,9690.200000,16.583130,5483.376471,9.188307,54.339931
std,275407.148728,63679.145467,10.220578,36057.610465,6.085087,6.209408
min,846.000000,64.000000,3.715826,34.000000,1.905090,28.302676
25%,5473.250000,683.250000,9.694484,347.500000,5.097916,50.191141
50%,12821.000000,1477.000000,13.902793,756.000000,7.337486,54.551823
75%,25938.500000,3960.500000,19.562850,2113.500000,11.331514,58.729329
max,3596677.000000,823867.000000,63.598326,466087.000000,32.887029,69.834711


### 1.c Town's with the largest share of Husky A enrollment vs. overall medicaid program

In [10]:
combined[combined["population"] > 5000].sort_values("husky_share", ascending=False).head(10)[["town","husky_share"]]

,town,husky_share
33,Danbury,64.918573
116,Redding,64.123711
71,Ledyard,64.096715
168,Woodstock,63.465160
21,Canterbury,63.010753
19,Burlington,63.008130
57,Griswold,62.915007
1,Ansonia,62.469257
87,Naugatuck,62.215772
101,North Stonington,61.983471


### 1.c Town's with the smallest share of Husky A enrollment vs. overall medicaid program


In [11]:
combined[combined["population"] > 5000].sort_values("husky_share", ascending=True).head(10)[["town","husky_share"]]

,town,husky_share
130,Southbury,28.302676
160,Wilton,39.492754
3,Avon,42.384106
155,Westbrook,42.469598
7,Bethany,43.198091
24,Cheshire,43.450479
51,Farmington,43.763756
118,Rocky Hill,43.889316
114,Prospect,44.585987
10,Bloomfield,44.646506


### 1.d Section conclusions

1. The lists with the highest medicaid enrollment and the highest husky A enrollment are virtually identical, with only New London and New Haven switching positions.
2. At first glance, the towns with the largest and smallest share of husky enrollment versus total medicaid enrollment seems random. Maybe it is just a proxy for the age of the population. Anecdotally, I think Southbury has a large elderly population.

## 2 Comparing age of town populations

In [12]:
#combined

In [15]:
age = pd.read_csv("ACS_14_5YR_S0101_with_ann.csv",
                 skiprows=1)
age = age[~age["Geography"].str.contains("County subdivisions not defined")]

age.dtypes

import pprint  
total_cols = [col for col in age.columns if 'Total; Estimate' in col and 'AGE' in col]
#pprint.pprint(total_cols)
#age_totals = age[total_cols]
#age_totals.dtypes

age_medians = age[["Geography","Total; Estimate; SUMMARY INDICATORS - Median age (years)"]]

age_groups = age[["Total; Estimate; AGE - Under 5 years",
     "Total; Estimate; SELECTED AGE CATEGORIES - 5 to 14 years",
     "Total; Estimate; SELECTED AGE CATEGORIES - 15 to 17 years"
    ]]

age_groups.loc[:,
               "under_5"] = pd.to_numeric(age_groups["Total; Estimate; AGE - Under 5 years"])
age_groups.loc[:,
               "5_to_14"] = pd.to_numeric(age_groups["Total; Estimate; SELECTED AGE CATEGORIES - 5 to 14 years"])
age_groups.loc[:,
               "15_to_17"] = pd.to_numeric(age_groups["Total; Estimate; SELECTED AGE CATEGORIES - 15 to 17 years"])
age_groups.loc[:,
             "under_18"] = age_groups["under_5"] + age_groups["5_to_14"] + age_groups["15_to_17"]

age_groups.loc[age["Geography"].notnull(),
                "town"] = age["Geography"].apply(lambda x: x[0:x.find(" town, ")])

age_under_18 = age_groups[["town", "under_18"]]

age_under_18.head()

,town,under_18
1,Bethel,22.8
2,Bridgeport,25.2
3,Brookfield,23.0
4,Danbury,21.2
5,Darien,33.7


In [14]:
#combined.head()


In [16]:
import numpy as np

age_medians.loc[age_medians["Geography"].notnull(),"town"] = age_medians["Geography"].apply(lambda x: x[0:x.find(" town, ")])
age_medians.loc[np.isreal(age_medians["Total; Estimate; SUMMARY INDICATORS - Median age (years)"]),
                                     "median_age"] = pd.to_numeric(age_medians["Total; Estimate; SUMMARY INDICATORS - Median age (years)"])
age_medians_clean = age_medians[["town","median_age"]]

#age_medians.sort_values(by="median_age", ascending=False)[["town","median_age"]]
age_medians_clean.describe()

,median_age
count,169.000000
mean,43.550888
std,4.877208
min,21.100000
25%,41.500000
50%,43.800000
75%,46.100000
max,58.600000


In [18]:
#combined = combined.merge(age_medians_clean,on="town")
combined = combined.merge(age_under_18,on="town")

In [19]:
combined.corr()

,population,med_enrollment,med_pct,husky_enrollment,husky_pct,husky_share,median_age,under_18
population,1.000000,0.891665,0.524308,0.890324,0.525807,0.086894,-0.580746,0.108752
med_enrollment,0.891665,1.000000,0.695984,0.998616,0.699091,0.171036,-0.559031,0.069922
med_pct,0.524308,0.695984,1.000000,0.696863,0.987432,0.278470,-0.473650,-0.203857
husky_enrollment,0.890324,0.998616,0.696863,1.000000,0.704848,0.195524,-0.561162,0.074034
husky_pct,0.525807,0.699091,0.987432,0.704848,1.000000,0.410273,-0.511964,-0.160799
husky_share,0.086894,0.171036,0.278470,0.195524,0.410273,1.000000,-0.312398,0.073939
median_age,-0.580746,-0.559031,-0.473650,-0.561162,-0.511964,-0.312398,1.000000,-0.183240
under_18,0.108752,0.069922,-0.203857,0.074034,-0.160799,0.073939,-0.183240,1.000000


In [20]:
combined[combined["population"] > 5000].sort_values("husky_share", ascending=False).head(10)[["town","husky_share","median_age"]]

,town,husky_share,median_age
33,Danbury,64.918573,36.8
116,Redding,64.123711,48.8
71,Ledyard,64.096715,41.4
167,Woodstock,63.465160,44.5
21,Canterbury,63.010753,43.4
19,Burlington,63.008130,42.4
57,Griswold,62.915007,37.3
1,Ansonia,62.469257,40.8
87,Naugatuck,62.215772,38.7
101,North Stonington,61.983471,46.9


In [21]:
combined[combined["population"] > 5000].sort_values("husky_share", ascending=True).head(10)[["town","husky_share","median_age"]]

,town,husky_share,median_age
130,Southbury,28.302676,50.1
160,Wilton,39.492754,42.2
3,Avon,42.384106,45.8
155,Westbrook,42.469598,48.9
7,Bethany,43.198091,44.6
24,Cheshire,43.450479,42.9
51,Farmington,43.763756,43.4
118,Rocky Hill,43.889316,43.4
114,Prospect,44.585987,45.0
10,Bloomfield,44.646506,48.2


### 2.a Section conclusions

1. There is no strong correlation between a town's median age and the share of Husky A enrollments compared with overall Medicaid enrollments.

## 3 Output for chart

In [21]:
combined.to_csv("output.csv")

,town,population,med_enrollment,med_pct,husky_enrollment,husky_pct,husky_share,median_age,under_18
0,Andover,3272,395,12.072127,226,6.907090,57.215190,43.0,23.3
1,Ansonia,18959,6099,32.169418,3810,20.095997,62.469257,40.8,22.8
2,Ashford,4259,888,20.849965,562,13.195586,63.288288,41.1,20.7
3,Avon,18421,1208,6.557733,512,2.779437,42.384106,45.8,25.1
4,Barkhamsted,3705,483,13.036437,283,7.638327,58.592133,44.7,22.0
5,Beacon Falls,6055,805,13.294798,431,7.118084,53.540373,43.0,22.1
6,Berlin,20610,2363,11.465308,1123,5.448811,47.524333,45.7,21.2
7,Bethany,5531,419,7.575484,181,3.272464,43.198091,44.6,24.0
8,Bethel,19372,2731,14.097667,1596,8.238695,58.440132,42.3,22.8
9,Bethlehem,3501,481,13.738932,278,7.940588,57.796258,48.9,20.8


In [43]:
import json

def hashed_json(dataset,index):
    dataset=json.loads(dataset.to_json(orient="records"))
    ret = {}
    for record in dataset:
        ret[record[index]] = record
        del (ret[record[index]][index])
    return ret

outfh = open ("medicaid.json","w")

outfh.write(json.dumps(hashed_json(combined,"town")))

outfh.close()

